#### Spark直接访问MySQL数据库  
spark可以直接访问MySQL业务数据库，获取数据，但是这样可能会导致业务数据库压力比较大，数据量较大时，不推荐这样处理

In [1]:
import os
# 配置pyspark和spark driver运行时 使用的python解释器
JAVA_HOME = '/root/bigdata/jdk'
PYSPARK_PYTHON = '/miniconda2/envs/py365/bin/python'
# 当存在多个版本时，不指定很可能会导致出错
os.environ['PYSPARK_PYTHON'] = PYSPARK_PYTHON
os.environ['PYSPARK_DRIVER_PYTHON'] = PYSPARK_PYTHON
os.environ['JAVA_HOME'] = JAVA_HOME
# 实现把mysql-connector-java-5.1.47.jar放入指定目录下
# 如果是使用spark直连MySQL数据库，读取数据，需要使用mysql链接的jar包，这里预先下载并放在了指定目录
os.environ["PYSPARK_SUBMIT_ARGS"] = "--jars /root/bigdata/spark/extraJars/mysql-connector-java-5.1.47.jar pyspark-shell"
# 配置spark信息
from pyspark import SparkConf
from pyspark.sql import SparkSession

SPARK_APP_NAME = "preprocessingMeiDuoData"
SPARK_URL = "spark://192.168.58.100:7077"

conf = SparkConf()    # 创建spark config对象
config = (
	("spark.app.name", SPARK_APP_NAME),    # 设置启动的spark的app名称，没有提供，将随机产生一个名称
	("spark.executor.memory", "2g"),    # 设置该app启动时占用的内存用量，默认1g，指一台虚拟机
	("spark.master", SPARK_URL),    # spark master的地址
    ("spark.executor.cores", "2"),    # 设置spark executor使用的CPU核心数，指一台虚拟机
    # 以下三项配置，可以控制执行器数量
#     ("spark.dynamicAllocation.enabled", True),
#     ("spark.dynamicAllocation.initialExecutors", 1),    # 1个执行器
#     ("spark.shuffle.service.enabled", True)
# 	('spark.sql.pivotMaxValues', '99999'),  # 当需要pivot DF，且值很多时，需要修改，默认是10000
)
# 查看更详细配置及说明：https://spark.apache.org/docs/latest/configuration.html

conf.setAll(config)

# 利用config对象，创建spark session
spark = SparkSession.builder.config(conf=conf).enableHiveSupport().getOrCreate()

In [4]:
# 直连访问mysql读取数据，这种方式数据没有经过hdfs，而是直接在MySQL数据库上进行操作，因此如果数据量较大，后续操作会对MySQL产生较大压力
# 因此该方式虽然用起来最爽，但是使用时一定要注意对mysql数据带来的影响
ret = spark.read.format("jdbc").options(url="jdbc:mysql://localhost/meiduo_mall",
                                       driver="com.mysql.jdbc.Driver",
                                       dbtable="tb_goods",
                                        user="root",
                                       password="password").load()#mysql的密码
ret

DataFrame[id: int, create_time: timestamp, update_time: timestamp, name: string, sales: int, comments: int, brand_id: int, category1_id: int, category2_id: int, category3_id: int, desc_detail: string, desc_pack: string, desc_service: string]

In [5]:
ret.show()

+---+--------------------+--------------------+--------------------+-----+--------+--------+------------+------------+------------+--------------------+--------------------+--------------------+
| id|         create_time|         update_time|                name|sales|comments|brand_id|category1_id|category2_id|category3_id|         desc_detail|           desc_pack|        desc_service|
+---+--------------------+--------------------+--------------------+-----+--------+--------+------------+------------+------------+--------------------+--------------------+--------------------+
|  1|2018-04-11 16:01:...|2018-04-25 12:09:...|Apple MacBook Pro...|    1|       1|       1|           4|          45|         157|<h1 style="text-a...|<h2>包装清单</h2>

...|<p>&nbsp;<strong>...|
|  2|2018-04-14 02:09:...|2018-04-25 11:51:...| Apple iPhone 8 Plus|    3|       1|       1|           1|          38|         115|<p><img alt="" sr...|<h3>包装清单</h3>

...|<p>&nbsp;<strong>...|
|  3|2018-04-14 03:03:...|201